# Modelado Predictivo de Tiempos de Inactividad

Este notebook desarrolla modelos predictivos para anticipar tiempos de inactividad en la línea de producción.

## Objetivos
1. Preparar datos para modelado predictivo
2. Desarrollar modelos de predicción
3. Evaluar y comparar modelos
4. Generar predicciones y recomendaciones

## Requisitos
- Python 3.8+
- Paquetes listados en requirements.txt
- Acceso a la base de datos de producción

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import sys
sys.path.append('..')

from utils.db_connection import create_connection, read_table

%matplotlib inline
plt.style.use('seaborn')

# Configurar reproducibilidad
np.random.seed(42)

## 1. Preparación de Datos

In [ ]:
# Cargar datos
conn = create_connection()
production_data = read_table(conn, 'manufacturing_line_productivity')
downtime_data = read_table(conn, 'normalizeddowntime')
factors_data = read_table(conn, 'downtime_factor')

# Preparar características
def prepare_features(prod_data, down_data, fact_data):
    # Agregar tiempo de inactividad total por lote
    batch_downtime = down_data.groupby('batch')['downtime_minutes'].sum().reset_index()
    
    # Unir con datos de producción
    df = pd.merge(prod_data, batch_downtime, on='batch', how='left')
    
    # Convertir fecha a datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Extraer características temporales
    df['weekday'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['hour'] = pd.to_datetime(df['Star_Time']).dt.hour
    
    # Codificar variables categóricas
    df = pd.get_dummies(df, columns=['flavor', 'size', 'operator'])
    
    return df

# Preparar dataset
model_data = prepare_features(production_data, downtime_data, factors_data)

# Separar características y objetivo
target = 'downtime_minutes'
features = [col for col in model_data.columns if col not in [target, 'date', 'Star_Time', 'End_Time', 'batch']]

X = model_data[features]
y = model_data[target]

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 2. Desarrollo del Modelo Base

In [ ]:
# Entrenar modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Evaluar modelo
y_pred = rf_model.predict(X_test_scaled)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Métricas del Modelo:")
print(f"RMSE: {rmse:.2f} minutos")
print(f"MAE: {mae:.2f} minutos")
print(f"R²: {r2:.3f}")

# Visualizar predicciones vs valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Tiempo de Inactividad Real (minutos)')
plt.ylabel('Tiempo de Inactividad Predicho (minutos)')
plt.title('Predicciones vs Valores Reales')
plt.tight_layout()
plt.savefig('../visualizations/predictions_vs_actual.png')
plt.show()

## 3. Análisis de Importancia de Características

In [ ]:
# Calcular importancia de características
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Visualizar importancia de características
plt.figure(figsize=(12, 6))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
plt.title('Top 15 Características más Importantes')
plt.xlabel('Importancia')
plt.tight_layout()
plt.savefig('../visualizations/feature_importance.png')
plt.show()

## 4. Predicciones para Nuevos Datos

In [ ]:
def predict_downtime(model, scaler, new_data):
    """
    Realiza predicciones para nuevos datos de producción.
    
    Args:
        model: Modelo entrenado
        scaler: Scaler ajustado
        new_data: DataFrame con nuevos datos de producción
    
    Returns:
        Predicciones de tiempo de inactividad
    """
    # Preparar nuevos datos
    prepared_data = prepare_features(new_data, pd.DataFrame(), pd.DataFrame())
    
    # Asegurar que tenemos todas las columnas necesarias
    missing_cols = set(features) - set(prepared_data.columns)
    for col in missing_cols:
        prepared_data[col] = 0
    
    # Ordenar columnas como en el entrenamiento
    prepared_data = prepared_data[features]
    
    # Escalar datos
    scaled_data = scaler.transform(prepared_data)
    
    # Realizar predicción
    predictions = model.predict(scaled_data)
    
    return predictions

# Ejemplo de uso con los últimos datos disponibles
recent_data = production_data.sort_values('date').tail(5)
predictions = predict_downtime(rf_model, scaler, recent_data)

print("\nPredicciones para los últimos 5 lotes:")
for batch, pred in zip(recent_data['batch'], predictions):
    print(f"Lote {batch}: {pred:.2f} minutos de inactividad predichos")

## 5. Validación Cruzada y Robustez del Modelo

In [ ]:
# Realizar validación cruzada
cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring='neg_root_mean_squared_error')
cv_rmse = -cv_scores

print("\nResultados de Validación Cruzada (RMSE):")
print(f"Media: {cv_rmse.mean():.2f} minutos")
print(f"Desviación Estándar: {cv_rmse.std():.2f} minutos")

# Visualizar distribución de errores
plt.figure(figsize=(10, 6))
residuals = y_test - y_pred
sns.histplot(residuals, kde=True)
plt.title('Distribución de Errores de Predicción')
plt.xlabel('Error (minutos)')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.savefig('../visualizations/prediction_errors.png')
plt.show()

## 6. Conclusiones y Recomendaciones

### Rendimiento del Modelo
1. Precisión:
   - RMSE: [Completar] minutos
   - R²: [Completar]
   - El modelo explica [X]% de la variabilidad en los tiempos de inactividad

### Factores Principales
1. Las características más influyentes son:
   - [Completar con top 3 características]
   - [Explicar implicaciones]

### Recomendaciones Operativas
1. Enfoque en Prevención:
   - [Completar con recomendaciones basadas en características importantes]
2. Monitoreo Continuo:
   - [Completar con estrategia de seguimiento]
3. Mejoras Sugeridas:
   - [Completar con acciones específicas]

## Próximos Pasos

1. Implementación en Producción:
   - Desarrollar API para predicciones en tiempo real
   - Integrar con sistemas existentes

2. Mejoras del Modelo:
   - Incorporar variables adicionales
   - Experimentar con otros algoritmos
   - Actualización periódica con nuevos datos

3. Validación Continua:
   - Monitorear rendimiento del modelo
   - Ajustar según retroalimentación
   - Documentar casos de éxito y áreas de mejora